# Vibrational spectra of Si

Here we compute the infrared and Raman spectra of silicon using `aiida-vibroscopy`.


## Finite electric fields

In the Placzek approximations (good for insulators), all the needed quantities for Raman can be computed as finite difference using the electric fields, as demonstrated in [the previous tutorial for the dielectric tensor](2_dielectric).
Now we a more challenging task, as a __third order derivative__ is needed: __the Raman tensor__, defined as:

\begin{equation}
    \frac{\partial \chi_{ij}}{\partial \tau_{K,k}} 
    =
    \frac{1}{\Omega}
    \frac{\partial^2 F_{K,k}}{\partial \mathcal{E}_i \partial \mathcal{E}_j}
\end{equation}

The same theory we saw in previous tutorials applies here. Note that for computing _second order_ derivatives of forces we need a slightly different formula:

\begin{equation}
    \left . \frac{\partial f(x)}{\partial x} \right|_{x=0}
    =
    \frac{1}{h^2}
    \left [
        f(h) -2f(0) +f(-2h)
    \right ]
    + \mathcal(O)(h^2)
\end{equation}

Have a look at the [finite difference coefficients](https://en.wikipedia.org/wiki/Finite_difference_coefficient) for coefficients of any accuracy order.

In [ ]:
from local_module import load_temp_profile

# If you download this file, you can run it with your own profile.
# Put these lines instead:
# from aiida import load_profile
# load_profile()
data = load_temp_profile(
    name="raman-tutorial",
    add_computer=True,
    add_pw_code=True,
    add_sssp=True,
)


## The `IRamanSpectraWorkChain`

For computing the spectra we need both phonons and Raman tensors.
Let's import the WorkChain and run it! We use the `get_builder_from_protocol` to get a __prefilled__ builder with __all inputs__. 

::: {note}
These inputs should be considered __not as converged parameters__, but as a good starting point. You may also need to tweak some parameters, e.g. add magnetization etc., depending on your case. 
:::

In [ ]:
from aiida.plugins import DbImporterFactory

CodDbImporter = DbImporterFactory('cod')

cod = CodDbImporter()
results = cod.query(id='1526655') # Si   1526655
structure = results[0].get_aiida_structure() # it has 8 atoms

In [ ]:
from aiida.plugins import WorkflowFactory
from aiida.engine import run_get_node

IRamanSpectraWorkChain = WorkflowFactory("vibroscopy.spectra.iraman")

builder = IRamanSpectraWorkChain.get_builder_from_protocol(
    code=data.pw_code,
    structure=structure,
    protocol="fast",
)

results, calc = run_get_node(builder)

These are the results:

In [ ]:
results

## The `VibrationalData`

In aiida-vibroscopy we design a data type able to store __all__ the phonon and dielectric properties, to ease the share of the data and make it as costistent as possible.

In this data, you can access to multiple information:

- Structure (cell, sites, ...)
- Symmetry
- Dielectric tensor ($\epsilon^{\infty}$)
- Born effective charges ($Z^*$)
- Raman tensors ($\partial \chi / \partial \tau$)
- Non-linear optical susceptibility ($\chi^{(2)}$)

Again, Born effective charges and non-linear optical susceptibility are null in silicon, due to symmetry. In the [next tutorial](3_polar) we will compute these quantities for AlAs, which has non vanishing tensors.

In [ ]:
vibro = calc.outputs.vibrational_data.numerical_accuracy_4
print("The VibrationalData: ", vibro, "\n")
print("The dielectric tensor: ", "\n", vibro.dielectric.round(5), "\n")
print("The Raman tensors (in 1/Angstrom): ", "\n", vibro.raman_tensors.round(5), "\n")

And we can also check how much the Raman tensors, which are third order derivatives, are sensible to the numerical accuracy chosen. Usually, the Raman tensor is expressed in Å$^2$. To get it, one can use the convention in the code:

\begin{equation}
    \left [
        \frac{\partial \chi}{\partial \tau}
    \right] (\text{Å}^2)
    = 
    \Omega_{unitcell}
    \left [
        \frac{\partial \chi}{\partial \tau}
    \right] (\text{Å}^{-1})
\end{equation}

In [ ]:
vol = calc.outputs.vibrational_data.numerical_accuracy_4.get_unitcell().get_cell_volume()

print(
    calc.outputs.vibrational_data.numerical_accuracy_4.raman_tensors[1,0,1,2]*vol,
    calc.outputs.vibrational_data.numerical_accuracy_2_step_1.raman_tensors[1,0,1,2]*vol,
    calc.outputs.vibrational_data.numerical_accuracy_2_step_2.raman_tensors[1,0,1,2]*vol,
)

We can see that the values are converged within 0.1 Å$^2$ with all three steps.

## Plotting the powder Raman spectra

Now, we can get directly from the data the Raman intensities.

In [ ]:
hh_intensities, hv_intensities, frequencies, labels = vibro.run_powder_raman_intensities(
    frequency_laser=532, temperature=300)
print(frequencies, "\n", labels)

We have 3 degenerate modes. Quite expected for silicon, which is very symmetric. To plot the actual spectra:

In [ ]:
from aiida_vibroscopy.utils.plotting import get_spectra_plot

total_intensities =  hh_intensities + hv_intensities
plt = get_spectra_plot(frequencies, total_intensities)
plt.show()

## Analysing the workflow

Look how the complexity of computing a Raman spectra has been handled __fully automatically__.

In [ ]:
%verdi process status {calc.pk}

## Convergence with k points

As a final remark, we show here how the convergence with k points is speeded up by sampling in this _directional_ manner, compared to the most 'traditional' uniform mesh. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Some options to make the plot nice
plt.rcParams['font.size'] = 18
plt.rcParams['font.family'] = "serif"
plt.rcParams['text.usetex'] = False
plt.rcParams['xtick.major.size'] = 7.0
plt.rcParams['xtick.minor.size'] = 4.
plt.rcParams['ytick.major.size'] = 7.0
plt.rcParams['ytick.minor.size'] = 4.
plt.rcParams['axes.linewidth'] = 1.2
plt.rcParams['legend.fontsize'] = 15

# Directional sampling
diel_new = np.array([12.333, 12.983, 13.182, 13.207])
kpoints_new = np.array([3*3*6, 3*3*12, 3*3*24, 3*3*58])

# Uniform sampling
diel = np.array([10.744, 12.369, 13.009])
kpoints = np.array([3*3*3, 6*6*6, 12*12*12])

# Plotting
fig, ax = plt.subplots(figsize=(6.5,5)) # dichiaro il canvas e i subplots    

ax.plot(kpoints_new, diel_new, color='red', marker='o', fillstyle='none', ms=10, linewidth=1.5, linestyle='-', label='Directional sampling')  
ax.plot(kpoints, diel, color='black', marker='o', fillstyle='none', ms=10, linewidth=1.5, linestyle='-', label='Uniform sampling')  
ax.set_xlabel('# k points')  
ax.set_ylabel(r'$\epsilon^{\infty}$')
ax.axhline(13.241, color='grey', linestyle='--', linewidth=1.5)
ax.legend(bbox_to_anchor=(0., 1.02, 1., .102), loc='lower left',
           ncol=1, mode="expand", borderaxespad=0., frameon=False)

In [ ]:
# Directional sampling
raman_new = np.array([18.445, 19.943, 20.442, 20.442])
kpoints_new = np.array([3*6*6, 3*12*12, 3*24*24, 3*58*58])

# Uniform sampling
raman = np.array([15.610, 18.762, 20.436])
kpoints = np.array([3*3*3, 6*6*6, 12*12*12])

# Plotting
fig, ax = plt.subplots(figsize=(6.5,5)) # dichiaro il canvas e i subplots    

ax.plot(kpoints_new, raman_new, color='red', marker='o', fillstyle='none', ms=10, linewidth=1.5, linestyle='-', label='Directional sampling')  
ax.plot(kpoints, raman, color='black', marker='o', fillstyle='none', ms=10, linewidth=1.5, linestyle='-', label='Uniform sampling')  
ax.set_xlabel('# k points')  
ax.set_ylabel(r'Raman tensor ($\AA^2$)')
ax.legend(bbox_to_anchor=(0., 1.02, 1., .102), loc='lower left',
           ncol=1, mode="expand", borderaxespad=0., frameon=False)